In [1]:
import pandas as pd 
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import CountVectorizer 
import numpy as np
from datetime import datetime
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder
from imblearn.over_sampling import SMOTE

In [2]:
dataset = pd.read_csv("/home/zeina/ryu/ryu/app/dataset-final.csv" , low_memory=False ,encoding='ISO 8859-1') 
df = pd.DataFrame(dataset)
df_orig=pd.DataFrame(dataset)
display(df)
print (df['priority'].value_counts(ascending=True))

,Device ID,Day,Start Time,End Time,URL,Keywords,type,content,priority
0,0,0,01:00,02:00,https://www.instagram.com/,Top accounts-hashtags-suggested accounts,browsing,social media,2
1,1,0,09:00,11:00,https://www.anghami.com/,ALL THE MUSIC YOU LOVE-Exclusive music-High qu...,audio streaming,entertainment,2
2,0,0,10:00,12:00,https://zoom.us/,Host-Whiteboard-Zoom Video Communications-Meet...,video streaming,education,5
3,0,0,13:00,13:30,https://www.skysports.com/football/news,Tables-Fixtures-Results-Roma-Transfers-Man Utd...,browsing,sports,1
4,3,0,13:00,13:20,https://www.youtube.com/watch?v=t0Q2otsqC4I,Tom-Jerry-Cartoon-Try Youtube Kids-Pizza Bean-...,video streaming,entertainment,3
...,...,...,...,...,...,...,...,...,...
775,0,6,17:30,18:00,https://www.instagram.com/,Hashtags- Instagram Lite- Instagram from Meta,browsing,social media,2
776,3,6,18:00,18:55,https://learnenglishkids.britishcouncil.org/,LearnEnglish -teaching-learn-learning English-...,browsing,education,4
777,4,6,20:15,23:00,https://www.filmdoo.com/films/wolpertinger/,Latest films-Category-Language-Submit a film-B...,video streaming,entertainment,4
778,5,6,22:30,23:00,https://www.instagram.com/,Hashtags- Instagram Lite- Instagram from Meta,browsing,social media,3


1     64
5    129
4    166
3    206
2    215
Name: priority, dtype: int64


In [3]:
df['day_of_week_sin'] = np.sin(df['Day'] * (2 * np.pi / 7))
df['day_of_week_cos'] = np.cos(df['Day'] * (2 * np.pi / 7))
df = df.drop('Day', 1)


/tmp/ipykernel_3306/3108558884.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop('Day', 1)


In [4]:
i=0
for x in df['Start Time'].values:
  x = datetime.strptime(x, '%H:%M')
  df['Start Time'].values[i]=x
  i=i+1

j=0
for y in df['End Time'].values:
  y = datetime.strptime(y, '%H:%M')
  df['End Time'].values[j]=y
  j=j+1


In [5]:
list=[]
for h in df['Start Time'].values:
    s=h.hour *60*60+ h.minute *60 
    list.append(s)
    #df['sin_time']=sin(2*np.pi*sec/seconds_in_day)

df['Start Time']=list

list2=[]
for h in df['End Time'].values:
    s=h.hour *60*60+ h.minute *60 
    list2.append(s)
    #df['sin_time']=sin(2*np.pi*sec/seconds_in_day)

df['End Time']=list2


In [6]:

seconds_in_day = 24*60*60


df['start_time_sin'] = np.sin(df['Start Time'] * (2 * np.pi / seconds_in_day))
df['start_time_cos'] = np.cos(df['Start Time'] * (2 * np.pi / seconds_in_day))

df['end_time_sin'] = np.sin(df['End Time'] * (2 * np.pi / seconds_in_day))
df['end_time_cos'] = np.cos(df['End Time'] * (2 * np.pi / seconds_in_day))

df = df.drop('Start Time', 1)
df = df.drop('End Time', 1)

display(df)

/tmp/ipykernel_3306/1628316626.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop('Start Time', 1)
/tmp/ipykernel_3306/1628316626.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop('End Time', 1)


,Device ID,URL,Keywords,type,content,priority,day_of_week_sin,day_of_week_cos,start_time_sin,start_time_cos,end_time_sin,end_time_cos
0,0,https://www.instagram.com/,Top accounts-hashtags-suggested accounts,browsing,social media,2,0.000000,1.00000,0.258819,9.659258e-01,5.000000e-01,8.660254e-01
1,1,https://www.anghami.com/,ALL THE MUSIC YOU LOVE-Exclusive music-High qu...,audio streaming,entertainment,2,0.000000,1.00000,0.707107,-7.071068e-01,2.588190e-01,-9.659258e-01
2,0,https://zoom.us/,Host-Whiteboard-Zoom Video Communications-Meet...,video streaming,education,5,0.000000,1.00000,0.500000,-8.660254e-01,1.224647e-16,-1.000000e+00
3,0,https://www.skysports.com/football/news,Tables-Fixtures-Results-Roma-Transfers-Man Utd...,browsing,sports,1,0.000000,1.00000,-0.258819,-9.659258e-01,-3.826834e-01,-9.238795e-01
4,3,https://www.youtube.com/watch?v=t0Q2otsqC4I,Tom-Jerry-Cartoon-Try Youtube Kids-Pizza Bean-...,video streaming,entertainment,3,0.000000,1.00000,-0.258819,-9.659258e-01,-3.420201e-01,-9.396926e-01
...,...,...,...,...,...,...,...,...,...,...,...,...
775,0,https://www.instagram.com/,Hashtags- Instagram Lite- Instagram from Meta,browsing,social media,2,-0.781831,0.62349,-0.991445,-1.305262e-01,-1.000000e+00,-1.836970e-16
776,3,https://learnenglishkids.britishcouncil.org/,LearnEnglish -teaching-learn-learning English-...,browsing,education,4,-0.781831,0.62349,-1.000000,-1.836970e-16,-9.713421e-01,2.376859e-01
777,4,https://www.filmdoo.com/films/wolpertinger/,Latest films-Category-Language-Submit a film-B...,video streaming,entertainment,4,-0.781831,0.62349,-0.831470,5.555702e-01,-2.588190e-01,9.659258e-01
778,5,https://www.instagram.com/,Hashtags- Instagram Lite- Instagram from Meta,browsing,social media,3,-0.781831,0.62349,-0.382683,9.238795e-01,-2.588190e-01,9.659258e-01


In [7]:
#vector=CountVectorizer().fit_transform(df['Keywords'])
#vector_Tfidf=TfidfTransformer().fit_transform(vector) 
#vect_df = pd.DataFrame(vector_Tfidf.todense())
#df=pd.concat([df, vect_df], axis=1)
df = df.drop('Keywords', 1)


/tmp/ipykernel_3306/1820012021.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop('Keywords', 1)


In [8]:
encoder = OrdinalEncoder()
result = encoder.fit_transform(df['type'].values.reshape(-1, 1))
df['type']=result
print(df['type'])



0      1.0
1      0.0
2      3.0
3      1.0
4      3.0
      ... 
775    1.0
776    1.0
777    3.0
778    1.0
779    1.0
Name: type, Length: 780, dtype: float64


In [9]:
encoder = OrdinalEncoder()
result = encoder.fit_transform(df['content'].values.reshape(-1, 1))
df['content']=result
print(df['content'])


0      7.0
1      2.0
2      0.0
3      8.0
4      2.0
      ... 
775    7.0
776    0.0
777    2.0
778    7.0
779    7.0
Name: content, Length: 780, dtype: float64


In [10]:
#vector=CountVectorizer().fit_transform(df['URL'])
#vector_Tfidf=TfidfTransformer().fit_transform(vector) 
#vect_df = pd.DataFrame(vector_Tfidf.todense())
#df=pd.concat([df, vect_df], axis=1)
df = df.drop('URL', 1)



/tmp/ipykernel_3306/1178753543.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop('URL', 1)


In [11]:
df1 = df.pop('priority') # remove column b and store it in df1
df['priority']=df1 # add b series as a 'new' column.



In [12]:
df.replace([np.inf, -np.inf], np.nan, inplace=True) #replace infinity values with NaN
df.dropna(inplace=True) #dropping rows with missing values  

count=df.shape[1]
x = df.iloc[:,:-1].values
y = df.iloc[:, count-1].values
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3)



In [13]:
counter = Counter(y_train) 
print('Before', counter) 
smtom = SMOTE() 
X_train_smtom, y_train_smtom = smtom.fit_resample (X_train, y_train)
counter = Counter(y_train_smtom) 
print('After', counter)


Before Counter({2: 155, 3: 152, 4: 115, 5: 87, 1: 37})
After Counter({4: 155, 3: 155, 1: 155, 2: 155, 5: 155})


In [14]:

rfc=RandomForestClassifier(random_state=42)
params = { 
    'n_estimators':  [100, 200, 300, 1000],
    'max_features': ['auto', 'sqrt'],
    'max_depth' : [4,5,6,7,8,10],
    'criterion' :['gini', 'entropy']
}

grid_search = GridSearchCV(estimator=rfc, 
                           param_grid=params, 
                           cv=4, n_jobs=-1, verbose=1, scoring = "accuracy")

grid_search.fit(X_train_smtom, y_train_smtom)
grid_search.best_params_



Fitting 4 folds for each of 96 candidates, totalling 384 fits


/home/zeina/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/zeina/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/zeina/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the defaul

{'criterion': 'entropy',
 'max_depth': 10,
 'max_features': 'auto',
 'n_estimators': 1000}

In [15]:

clf=RandomForestClassifier(n_estimators=1000,criterion='entropy',max_depth=10,max_features='auto')
clf = clf.fit(X_train_smtom, y_train_smtom)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))


/home/zeina/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


              precision    recall  f1-score   support

           1       0.92      0.81      0.86        27
           2       0.85      0.97      0.91        60
           3       0.94      0.89      0.91        54
           4       0.98      0.88      0.93        51
           5       0.93      1.00      0.97        42

    accuracy                           0.92       234
   macro avg       0.92      0.91      0.92       234
weighted avg       0.92      0.92      0.92       234



In [16]:
def predict(device_id,day,start_time,server):
    
    Dev_id=int(device_id)
    D=int(day)
    S_time=start_time
    server_name=server
    i=0
    url=None
    indices=[]
    for x in df_orig['URL'].values:
        if server_name in x:
            url=x
            indices.append(i)
            x=df.iloc[:,:-1].values[i]
            #print(x)

        i=i+1

    #print(indices)
    print("URL",url)
    print('Device ID',Dev_id)
    print('Day',D)
    print('Start time',S_time)
    
    
    index = random.choice(tuple(indices))
    #print(index)
    row=df.iloc[:,:-1].values[index]
    #print("row",row)
    day_sin = np.sin(D * (2 * np.pi / 7))
    day_cos = np.cos(D * (2 * np.pi / 7))
    #print(day_sin)
    row[0]=Dev_id
    row[3]=day_sin
    row[4]=day_cos
    
    splitted_start_time = S_time.split(':')

    hour_start=int(splitted_start_time[0])
    hour_end=hour_start+1

    sec_start=hour_start *60*60
    sec_end=hour_start *60*60

    seconds_in_day = 24*60*60

    start_time_sin = np.sin(sec_start * (2 * np.pi / seconds_in_day))
    start_time_cos = np.cos(sec_start * (2 * np.pi / seconds_in_day))

    end_time_sin = np.sin(sec_end * (2 * np.pi / seconds_in_day))
    end_time_cos = np.cos(sec_end * (2 * np.pi / seconds_in_day))

    row[4]=start_time_sin
    row[5]=start_time_cos
    row[6]=end_time_sin
    row[7]=end_time_cos
    priority=clf.predict([row])
    return priority


In [22]:

from json import dumps
from flask import Flask,request,jsonify,make_response
import random
Dev_id=None 
D=None
S_time=None
server_name=None
print('start')  
app = Flask(__name__)
@app.route('/api',methods=['POST'])
def getparam():
    data = request.get_json(force=True)
    Dev_id=data['device_id'] 
    D=data['day']
    S_time=data['start_time'] 
    server_name=data['server'] 
    priority_value=predict(Dev_id,D,S_time,server_name)
    print('priority=',priority_value)
    priority_dict = dict()
    for index,value in enumerate(priority_value):
        priority_dict[index] = value

    return make_response(dumps(priority_value.tolist()))

app.run()         


start
 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000 (Press CTRL+C to quit)


URL https://www.instagram.com/
Device ID 0
Day 2
Start time 11:21


127.0.0.1 - - [27/Jun/2022 11:21:19] "POST /api HTTP/1.1" 200 -


priority= [3]
